# ETA Prediction Pipeline Report

## Introduction

This notebook implements a pipeline to predict the Expected Time of Arrival (ETA) for ride requests. The ETA is decomposed into three key components, each predicted by a separate model:

1.  **$P_a$ (Driver Search Time):** The estimated time it takes for a driver to accept the ride request. This is heavily influenced by the local supply-to-demand ratio (SDR).
2.  **$P_b$ (Driver-to-Pickup Time):** The estimated time for the assigned driver to travel to the user's pickup location. This is also influenced by the local SDR.
3.  **$P_c$ (Trip Duration):** The estimated time for the actual ride from the pickup point to the destination. This is calculated using road network data and historical average speeds.

The pipeline processes a CSV of ride requests, enriches it with geospatial data (SDR and road speeds), applies the models, and outputs the predicted $P_a, P_b,$ and $P_c$ values in a JSON file.

### 1. Setup and Imports

First, we import all the necessary libraries. This includes `pandas` for data manipulation, `h3` for hexagonal geospatial indexing, `osmnx` and `networkx` for road network analysis, and others for various utility functions.

In [ ]:
import argparse
import json
import os
import pandas as pd
import numpy as np
import math
import h3
import ast
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, Polygon, box
from shapely.ops import transform
from pyproj import CRS, Transformer

# Set pandas display options for better viewing
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### 2. Configuration and Model Parameters

These parameters control the behavior of the models and the data processing steps. They are centralized here for easy tuning and experimentation.

In [2]:
# --- General Configuration ---
# Set to True to use a constant 10 minutes for pc, or False to calculate it dynamically.
USE_CONSTANT_PC = False 

# --- Heuristic Model Parameters ---
# H3 is a geospatial indexing system that divides the world into hexagons. Resolution 7 is a good balance for city-level analysis.
H3_RES = 7
# The radius around a request point to consider for local supply/demand.
CIRCLE_RADIUS_M = 1840.0

# --- Pa Model Parameters (Driver Search Time) ---
# Best parameters found from testing for the Pa logistic function
PA_ALPHA = 2.0
PA_BETA = 0.0

# --- Pb Model Parameters (Driver-to-Pickup Time) ---
# Parameters for the Pb exponential model
PB_ALPHA = 0.2
PB_BETA = 0

# --- Pc Model Parameters (Trip Duration) ---
# Fallback constant for trip time (pc) in minutes
PC_CONST = 10.0

# --- Logic Thresholds ---
# SDR value below which a ride is considered a "failed attempt" with high Pa
SDR_FAILURE_THRESHOLD = 0.1

### 3. File Paths

In a notebook environment, we define the input and output file paths directly, replacing the command-line argument parser from the original script.

In [ ]:
# Define paths for local execution
DEFAULT_INPUT_CSV = "data/input.csv"
DEFAULT_OUTPUT_JSON = "out/output.json"
DEFAULT_SDR_PATH = "ref_data/processed_hex_avg_SDR.parquet"
DEFAULT_SPEED_PATH = "ref_data/smoothed_speed_full.parquet"

# Check if required directories exist, create if not
os.makedirs("data", exist_ok=True)
os.makedirs("out", exist_ok=True)
os.makedirs("ref_data", exist_ok=True)

print("Please ensure the following files are placed correctly:")
print(f"Input ride data: '{DEFAULT_INPUT_CSV}'")
print(f"SDR reference data: '{DEFAULT_SDR_PATH}'")
print(f"Speed reference data: '{DEFAULT_SPEED_PATH}'")

### 4. Utility Functions

This section contains the helper functions used throughout the pipeline for tasks like time conversion, coordinate parsing, and geospatial calculations.

#### 4.1. Data Parsing and Conversion

Functions to process raw data from the input files.

In [ ]:
def time_to_slot(timestr):
    """Converts a 'HH:MM:SS' string to a 15-minute time slot index (0-95)."""
    try:
        hh, mm, ss = [int(x) for x in timestr.split(":")]
    except Exception:
        return 0
    total_minutes = hh * 60 + mm
    return min(max(total_minutes // 15, 0), 95)

def parse_coordinates(s):
    """Parses a string like '(lat, lon)' into a tuple of floats."""
    if pd.isna(s):
        return (None, None)
    # Strip parentheses and quotes before splitting
    ss = str(s).strip().replace("(", "").replace(")", "").replace('"', '')
    parts = [p.strip() for p in ss.split(",")]
    
    if len(parts) != 2:
        return (None, None)
    try:
        # Latitude is first, Longitude is second
        return float(parts[0]), float(parts[1])
    except Exception:
        return (None, None)

def convert_h3_to_int64(h3_hex_str):
    """Converts a hexadecimal H3 string to a 64-bit integer, safely handling invalid inputs."""
    if pd.isna(h3_hex_str) or h3_hex_str is None:
        return np.int64(0)
    try:
        return np.int64(int(str(h3_hex_str), 16))
    except ValueError:
        return np.int64(0)

#### 4.2. Geospatial and H3 Functions

These functions handle the core geospatial logic. We use an Azimuthal Equidistant (AEQD) projection to accurately calculate areas and distances in meters. The `compute_hex_circle_weights` function is crucial: it finds all H3 hexagons that overlap with a circle of a given radius around the ride request's origin and calculates the percentage of overlap. This weighted area is used to calculate a weighted average SDR.

In [ ]:
def get_h3_index(lat, lon, res):
    """Finds the H3 hexagon index for a given lat/lon and resolution."""
    return h3.latlng_to_cell(lat, lon, res)

def build_aeqd_transformer(lat_center, lon_center):
    """Creates projection transformers for accurate distance calculations."""
    aeqd_proj4 = f"+proj=aeqd +lat_0={lat_center} +lon_0={lon_center} +units=m +datum=WGS84 +no_defs"
    aeqd_crs = CRS.from_proj4(aeqd_proj4)
    wgs84 = CRS.from_epsg(4326)
    to_proj = Transformer.from_crs(wgs84, aeqd_crs, always_xy=True)
    to_wgs84 = Transformer.from_crs(aeqd_crs, wgs84, always_xy=True)
    return to_proj.transform, to_wgs84.transform

def compute_hex_circle_weights(lat, lon, radius_m=CIRCLE_RADIUS_M, h3_res=H3_RES, k_ring_k=3):
    """
    Calculates the proportional overlap of H3 hexagons within a specified radius
    of a central point. This is used to create a weighted SDR average.
    """
    center_h3_hexadecimal = get_h3_index(lat, lon, h3_res)
    if not center_h3_hexadecimal:
        return []
        
    # Get candidate hexagons in the vicinity
    candidates = h3.grid_disk(center_h3_hexadecimal, k_ring_k)

    # Project circle to a coordinate system where distance is accurate (in meters)
    to_proj, _ = build_aeqd_transformer(lat, lon)
    cx, cy = to_proj(lon, lat)
    circle_proj = Point(cx, cy).buffer(radius_m, resolution=64)

    weights = []
    for h_hex_str in candidates: 
        h3_index_int64 = convert_h3_to_int64(h_hex_str)
        hex_coords = [(lng, lat) for lat, lng in h3.cell_to_boundary(h_hex_str)]
        hex_poly_wgs = Polygon(hex_coords)
        hex_poly_proj = transform(lambda x, y: to_proj(x, y), hex_poly_wgs)

        # Calculate the intersection area and the corresponding weight
        inter = hex_poly_proj.intersection(circle_proj)
        inter_area = inter.area if not inter.is_empty else 0.0
        hex_area = hex_poly_proj.area if hex_poly_proj.area > 0 else 1.0
        weight = inter_area / hex_area

        if weight > 0:
            weights.append((h3_index_int64, weight))
            
    return weights

### 5. ETA Component Models

These are the core predictive models for each component of the ETA.

#### 5.1. $P_a$ and $P_b$ Heuristic Models

$P_a$ (driver search time) and $P_b$ (driver-to-pickup time) are predicted using heuristic models based on the weighted average Supply-to-Demand Ratio (SDR) in the area of the ride request. The logic is that higher SDR (more drivers than riders) leads to shorter wait times.

The $P_a$ model uses a logistic function. The probability $p$ is calculated as:
$$p = \frac{1}{1 + e^{-(\alpha \cdot \log(1+y) + \beta)}}$$
Where $y$ is the weighted SDR. The final prediction is $P_a = 9 \cdot (1 - p)$.

In [ ]:
def lookup_sdr(sdr_series, h3_index, slot, fallback=0.0):
    """Efficiently looks up the SDR value from a pre-processed series."""
    try:
        return float(sdr_series.loc[(h3_index, slot)])
    except KeyError:
        return fallback

def predict_pa_robust(SDR_list, area_list, alpha=PA_ALPHA, beta=PA_BETA):
    """Predicts Pa (driver search time) based on weighted SDR."""
    SDR_list = np.array(SDR_list)
    area_list = np.array(area_list)
    weighted_sdr = np.sum(SDR_list * area_list)
    
    # If SDR is very low, assume a failed attempt and return a high wait time
    if weighted_sdr < SDR_FAILURE_THRESHOLD:
        return 30.0
    else:
        log_transformed_sdr = math.log(1 + weighted_sdr)
        prob = 1 / (1 + np.exp(-(alpha * log_transformed_sdr + beta)))
        predicted_pa = 9.0 * (1 - prob)
        return predicted_pa

def predict_pb_exponential(SDR_list, area_list, alpha=PB_ALPHA, beta=PB_BETA):
    """Predicts Pb (driver-to-pickup time) based on weighted SDR."""
    SDR_list = np.array(SDR_list)
    area_list = np.array(area_list)
    y = np.sum(SDR_list * area_list)
    prob = 1 / (1 + np.exp(-(alpha * y + beta)))
    return 7 * (1 - prob)

#### 5.2. $P_c$ Dynamic Trip Time Model

The trip time, $P_c$, is calculated by finding the shortest path on the actual road network using `OSMnx`. The travel time for each road segment in the path is estimated using a pre-computed lookup table of historical average speeds for that segment and time of day. This provides a much more accurate estimate than simple straight-line distance.

In [3]:
def edge_speed_in_df(u, v, speed_df, timeslot_col, G):
    """Looks up the average speed for a given road segment (u, v) at a specific time."""
    def mean_speed(df):
        return float(df[timeslot_col].mean()) if not df.empty else None

    # Try to find an exact match for the edge
    match = speed_df[((speed_df['u'] == u) & (speed_df['v'] == v)) | ((speed_df['u'] == v) & (speed_df['v'] == u))]
    if not match.empty: return mean_speed(match)

    # Fallback to matching by OpenStreetMap ID
    edge_data_dict = G.get_edge_data(u, v)
    osmid_set = set()
    if edge_data_dict:
        for _, attr in edge_data_dict.items():
            osmid = attr.get('osmid')
            if isinstance(osmid, list): osmid_set.update(osmid)
            elif osmid is not None: osmid_set.add(osmid)
    if osmid_set:
        match = speed_df[speed_df['osmid'].isin(osmid_set)]
        if not match.empty: return mean_speed(match)
        
    return np.nan

def predict_pc_dynamic_graph(start_lat, start_lon, end_lat, end_lon, time_str, speed_df, default_speed_kph=19.5):
    """
    Calculates route travel time by downloading the street network for the ride's
    bounding box, finding the shortest path, and summing segment travel times.
    """
    try:
        # Determine the time slot for speed lookup
        hh, mm, ss = [int(x) for x in time_str.split(":")]
        seconds_from_midnight = hh * 3600 + mm * 60 + ss
        timeslot = str(seconds_from_midnight // 900)

        # Download the relevant section of the map
        buffer_deg = 0.01
        north, south = max(start_lat, end_lat) + buffer_deg, min(start_lat, end_lat) - buffer_deg
        east, west = max(start_lon, end_lon) + buffer_deg, min(start_lon, end_lon) - buffer_deg
        G = ox.graph_from_bbox(north, south, east, west, network_type="drive_service", simplify=True)

        # Find the nearest network nodes to the start/end points and calculate the route
        start_node, end_node = ox.nearest_nodes(G, X=(start_lon, end_lon), Y=(start_lat, end_lat))
        route_nodes = nx.shortest_path(G, source=start_node, target=end_node, weight='length')
        
        total_time_sec = 0
        last_speed = default_speed_kph
        
        # Iterate over segments in the route to calculate total time
        for i in range(len(route_nodes) - 1):
            u, v = route_nodes[i], route_nodes[i+1]
            avg_speed = edge_speed_in_df(u, v, speed_df, timeslot, G)
            
            if pd.isna(avg_speed) or avg_speed < 0.5:
                avg_speed = last_speed
            
            last_speed = avg_speed
            edge_length_m = G.get_edge_data(u, v, key=0).get('length', 0)
            
            if avg_speed > 0:
                time_sec = (edge_length_m / 1000) / avg_speed * 3600
                total_time_sec += time_sec
                
        return total_time_sec / 60
    
    except Exception as e:
        print(f"PC WARNING: Dynamic graph calculation failed (Error: {e}). Using fallback PC={PC_CONST}.")
        return PC_CONST

### 6. Main Execution Pipeline

This is the main part of the notebook where we load the data, process each ride request one by one, and generate the final predictions.

#### 6.1. Load and Prepare Data

We load the input CSV with ride requests and the Parquet files containing the SDR and speed data. The SDR data is then "melted" from a wide format (one column per time slot) to a long format, which allows for much faster lookups using a multi-index.

In [ ]:
# --- Load Data ---
try:
    df_rides = pd.read_csv(DEFAULT_INPUT_CSV, dtype=str)
    print(f"SUCCESS: Input CSV ({len(df_rides)} rows) loaded.")
    display(df_rides.head())
    
    sdr_df = pd.read_parquet(DEFAULT_SDR_PATH)
    print(f"SUCCESS: SDR data ({len(sdr_df)} hexes) loaded.")
    display(sdr_df.head())
    
except Exception as e:
    print(f"FATAL ERROR: Could not load core data files. Please check paths. Error: {e}")

# --- Pre-process SDR Data for efficient lookup ---
slot_columns = [f'slot_{i}' for i in range(96)]
sdr_long_df = sdr_df.melt(id_vars=['h3_index'], value_vars=slot_columns, var_name='slot_name', value_name='SDR')
sdr_long_df['slot'] = sdr_long_df['slot_name'].str.split('_').str[-1].astype(int)
sdr_long_df['h3_index'] = sdr_long_df['h3_index'].astype(np.int64)
sdr_series = sdr_long_df.set_index(["h3_index", "slot"])["SDR"]
print(f"SUCCESS: SDR data prepared for lookup.")

# --- Load Speed Data ---
speed_df = None
if os.path.exists(DEFAULT_SPEED_PATH):
    try:
        speed_df = pd.read_parquet(DEFAULT_SPEED_PATH)
        if 'u' in speed_df.columns: speed_df['u'] = speed_df['u'].astype(np.int64)
        if 'v' in speed_df.columns: speed_df['v'] = speed_df['v'].astype(np.int64)
        print(f"SUCCESS: Speed Data loaded from '{DEFAULT_SPEED_PATH}'.")
    except Exception as e:
        print(f"WARNING: Speed Data load FAILED: {e}. PC will use fallback.")
else:
    print(f"WARNING: Speed Data file NOT FOUND. PC will use fallback.")

#### 6.2. Process Each Ride

We iterate through each row of the input DataFrame, apply the functions and models defined above, and store the results in a dictionary.

In [ ]:
results = {}
print("\nStarting prediction for each ride...")

for index, row in df_rides.iterrows():
    rid = row["rid"]
    print(f"--- Processing rid: {rid} ---")
    
    # 1. Parse inputs
    slot = time_to_slot(row["ride_request_time"])
    lat, lon = parse_coordinates(row["ride_start_location"])
    end_lat, end_lon = parse_coordinates(row["ride_end_location"])

    if lat is None or lon is None or end_lat is None or end_lon is None:
        results[rid] = {"pa": 0.0, "pb": 0.0, "pc": PC_CONST}
        print(f"  -> Invalid coordinates. Skipping.")
        continue

    # 2. Get weighted SDR for Pa and Pb models
    weights = compute_hex_circle_weights(lat, lon)
    if not weights:
        # Fallback to just the center hex if circle calculation fails
        center_h3_hex = get_h3_index(lat, lon, H3_RES)
        center_h3_int = convert_h3_to_int64(center_h3_hex)
        if center_h3_int != np.int64(0):
            weights = [(center_h3_int, 1.0)]
        else:
            results[rid] = {"pa": 0.0, "pb": 0.0, "pc": PC_CONST}
            print(f"  -> Invalid H3 index. Skipping.")
            continue
            
    SDRs, areas = [], []
    for h, w in weights:
        SDRs.append(lookup_sdr(sdr_series, h, slot))
        areas.append(w)

    # 3. Predict Pa and Pb
    pa = predict_pa_robust(SDRs, areas)
    pb = predict_pb_exponential(SDRs, areas)
    print(f"  -> Pa={pa:.2f}, Pb={pb:.2f}")

    # 4. Predict Pc
    if USE_CONSTANT_PC:
        pc = PC_CONST
    elif speed_df is not None:
        pc = predict_pc_dynamic_graph(lat, lon, end_lat, end_lon, row["ride_request_time"], speed_df)
    else:
        pc = PC_CONST # Fallback if speed data is missing
    print(f"  -> Pc={pc:.2f}")

    # 5. Store results
    results[rid] = {"pa": round(pa, 2), "pb": round(pb, 2), "pc": round(pc, 2)}
    
print("\n...Processing complete.")

### 7. Save and Display Results

Finally, we save the `results` dictionary to the specified output JSON file and also display the first few results as a DataFrame for a quick review.

In [ ]:
# Save results to a JSON file
with open(DEFAULT_OUTPUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"\nSUCCESS: Results saved to {DEFAULT_OUTPUT_JSON}")

# Display results in a DataFrame for review
results_df = pd.DataFrame.from_dict(results, orient='index')
print("\n--- Final Predictions ---")
display(results_df)